In [1]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import array_to_img
import cv2
#from data import *

from keras.preprocessing.image import img_to_array, load_img
import numpy as np
import glob
import tensorflow

In [2]:
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
import glob


class dataProcess(object):
    def __init__(self, out_rows, out_cols, data_path=".\\data\\train\\images", label_path=".\\data\\train\\labels",
                 test_path=".\\data\\test\\images", npy_path=".\\data\\npydata", img_type="jpg"):
        self.out_rows = out_rows
        self.out_cols = out_cols
        self.data_path = data_path
        self.label_path = label_path
        self.img_type = img_type
        self.test_path = test_path
        self.npy_path = npy_path

    def create_train_data(self):
        i = 0
        print('Creating training images...')
        imgs = glob.glob(self.data_path+"\\*."+self.img_type)
        print(len(imgs))
        imgdatas = np.ndarray((len(imgs), self.out_rows, self.out_cols, 1), dtype=np.uint8)
        imglabels = np.ndarray((len(imgs), self.out_rows, self.out_cols, 1), dtype=np.uint8)

        for x in range(len(imgs)):
            imgpath = imgs[x]
            pic_name = imgpath.split('\\')[-1]
            labelpath = self.label_path + '\\' + pic_name
            img = load_img(imgpath, grayscale=True, target_size=[320, 320])
            label = load_img(labelpath, grayscale=True, target_size=[320, 320])
            img = img_to_array(img)
            label = img_to_array(label)
            imgdatas[i] = img
            imglabels[i] = label
            if i % 100 == 0:
                print('Done:', len(imgs),' images')
            i += 1

        print('loading done')
        np.save(self.npy_path + '.\\imgs_train.npy', imgdatas)
        np.save(self.npy_path + '.\\imgs_mask_train.npy', imglabels)
        print('Saving to .npy files done.')

    def create_test_data(self):
        i = 0
        print('Creating test images...')
        imgs = glob.glob(self.test_path + "\\*." + self.img_type)
        imgdatas = np.ndarray((len(imgs), self.out_rows, self.out_cols, 1), dtype=np.uint8)
        testpathlist = []

        for imgname in imgs:
            testpath = imgname
            testpathlist.append(testpath)
            img = load_img(testpath, grayscale=True, target_size=[320, 320])
            img = img_to_array(img)
            imgdatas[i] = img
            i += 1

        txtname = '.\\data\\results\\pic.txt'
        with open(txtname, 'w') as f:
            for i in range(len(testpathlist)):
                f.writelines(testpathlist[i] + '\n')
        print('loading done')
        np.save(self.npy_path + '\\imgs_test.npy', imgdatas)
        print('Saving to imgs_test.npy files done.')

    def load_train_data(self):
        print('load train images...')
        imgs_train = np.load(self.npy_path + "\\imgs_train.npy")
        imgs_mask_train = np.load(self.npy_path + "\\imgs_mask_train.npy")
        imgs_train = imgs_train.astype('float32')
        imgs_mask_train = imgs_mask_train.astype('float32')
        imgs_train /= 255
        imgs_mask_train /= 255
        #imgs_mask_train[imgs_mask_train > 0.5] = 1  
        #imgs_mask_train[imgs_mask_train <= 0.5] = 0 
        return imgs_train, imgs_mask_train

    def load_test_data(self):
        print('-' * 30)
        print('load test images...')
        print('-' * 30)
        imgs_test = np.load(self.npy_path + "\\imgs_test.npy")
        imgs_test = imgs_test.astype('float32')
        imgs_test /= 255
        return imgs_test

In [16]:
mydata = dataProcess(320, 320)
mydata.create_train_data()
mydata.create_test_data()

Creating training images...
2996
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
Done: 2996  images
loading done
Saving to .npy files done.
Creating test images...
loading done
Saving to imgs_test.npy files done.


In [3]:
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [8]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import array_to_img
import cv2
from data import *
import tensorflow

def batch_ssim(y_true, y_pred):
    return tensorflow.reduce_mean(tensorflow.image.ssim(y_true, y_pred, max_val=1.0))


def batch_msssim(y_true, y_pred):
    return tensorflow.reduce_mean(tensorflow.image.ssim_multiscale(y_true, y_pred, max_val=1.0))


def batch_psnr(y_true, y_pred):
    return tensorflow.reduce_mean(tensorflow.image.psnr(y_true, y_pred, max_val=1.0))


def batch_nmse(y_true, y_pred):
    return tensorflow.divide(tensorflow.reduce_sum(tensorflow.math.squared_difference(y_true, y_pred)), tensorflow.reduce_sum(y_true ** 2))


def loss_func(labels, predictions):
        return 1 - tensorflow.image.ssim(labels, predictions, max_val=15.0)


class myUnet(object):
    def __init__(self, img_rows=320, img_cols=320):
        self.img_rows = img_rows
        self.img_cols = img_cols

    def load_data(self):
        mydata = dataProcess(self.img_rows, self.img_cols)
        imgs_train, imgs_mask_train = mydata.load_train_data()
        imgs_test = mydata.load_test_data()
        return imgs_train, imgs_mask_train, imgs_test

    def get_unet(self):
        inputs = Input((self.img_rows, self.img_cols, 1))

        conv1 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
        # print(conv1)
        conv1 = BatchNormalization()(conv1)
        print ("conv1 shape:", conv1.shape)
        conv1 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
        conv1 = BatchNormalization()(conv1)
        print ("conv1 shape:", conv1.shape)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        print ("pool1 shape:", pool1.shape)

        conv2 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
        print ("conv2 shape:", conv2.shape)
        conv2 = BatchNormalization()(conv2)
        conv2 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
        print ("conv2 shape:", conv2.shape)
        conv2 = BatchNormalization()(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        print ("pool2 shape:", pool2.shape)

        conv3 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
        print ("conv3 shape:", conv3.shape)
        conv3 = BatchNormalization()(conv3)
        conv3 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
        print ("conv3 shape:", conv3.shape)
        conv3 = BatchNormalization()(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        print ("pool3 shape:", pool3.shape)

        conv4 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
        conv4 = BatchNormalization()(conv4)
        conv4 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
        conv4 = BatchNormalization()(conv4)
        drop4 = Dropout(0.5)(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

        conv5 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
        conv5 = BatchNormalization()(conv5)
        #conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
       # conv5 = BatchNormalization()(conv5)
        drop5 = Dropout(0.5)(conv5)

        up6 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(drop5))
        up6 = BatchNormalization()(up6)
        merge6 = concatenate([drop4, up6], axis=3)
        print(up6)
        print(merge6)
        conv6 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
        print(conv6)
        conv6 = BatchNormalization()(conv6)
        conv6 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)
        print(conv6)
        conv6 = BatchNormalization()(conv6)
        up7 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
            UpSampling2D(size=(2, 2))(conv6))
        up7 = BatchNormalization()(up7)
        merge7 = concatenate([conv3, up7], axis=3)
        print(up7)
        print(merge7)
        conv7 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
        conv7 = BatchNormalization()(conv7)
        print(conv7)
        conv7 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)
        print(conv7)
        conv7 = BatchNormalization()(conv7)
        up8 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
            UpSampling2D(size=(2, 2))(conv7))
        up8 = BatchNormalization()(up8)
        merge8 = concatenate([conv2, up8], axis=3)
        conv8 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
        conv8 = BatchNormalization()(conv8)
        conv8 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)
        conv8 = BatchNormalization()(conv8)
        up9 = Conv2D(32, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
            UpSampling2D(size=(2, 2))(conv8))
        up9 = BatchNormalization()(up9)
        merge9 = concatenate([conv1, up9], axis=3)
        print(up9)
        print(merge9)
        conv9 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
        conv9 = BatchNormalization()(conv9)
        print(conv9)
        conv9 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
        conv9 = BatchNormalization()(conv9)
        print(conv9)
        conv9 = Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
        conv9 = BatchNormalization()(conv9)
        print ("conv9 shape:", conv9.shape)

        conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)
        print(conv10)
        model = Model(inputs=inputs, outputs=conv10)

        model.compile(optimizer=RMSprop(lr=0.01), loss=loss_func, metrics=[batch_ssim, batch_psnr, batch_msssim, batch_nmse])
        model.summary()
        return model

    def train(self):
        print("loading data")
        imgs_train, imgs_mask_train, imgs_test = self.load_data()
        print("loading data done")
        model = self.get_unet()
        print("got unet")
        model_checkpoint = ModelCheckpoint('unet.hdf5', monitor='loss', verbose=1, save_best_only=True)
        print('Fitting model...')
        model.fit(imgs_train, imgs_mask_train, batch_size=2, epochs=40, verbose=1,
                  validation_split=0.05, shuffle=True, callbacks=[model_checkpoint])
        model.save_weights('./unet_model.hdf5')
        print('predict test data')
        imgs_mask_test = model.predict(imgs_test, batch_size=1, verbose=1)
        np.save('./data/results/imgs_mask_test.npy', imgs_mask_test)

    def save_img(self):
        print("array to image")
        imgs = np.load('./data/results/imgs_mask_test.npy')
        piclist = []
        for line in open("./data/results/pic.txt"):
            line = line.strip()
            picname = line.split('/')[-1]
            piclist.append(picname)
        print(len(piclist))
        for i in range(imgs.shape[0]):
            path = "./data/results/" + piclist[i]
            img = imgs[i]
            img = array_to_img(img)
            img.save(path)
            cv_pic = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            cv_pic = cv2.resize(cv_pic,(1918,1280),interpolation=cv2.INTER_CUBIC)
            #binary, cv_save = cv2.threshold(cv_pic, 127, 255, cv2.THRESH_BINARY)
            cv2.imwrite(path, cv_pic)
            
    def load_model_weights(self, model):
        model.load_weights('./unet_model.hdf5')

In [ ]:
myunet = myUnet()
model = myunet.get_unet()
myunet.train()
myunet.save_img()

conv1 shape: (None, 320, 320, 32)
conv1 shape: (None, 320, 320, 32)
pool1 shape: (None, 160, 160, 32)
conv2 shape: (None, 160, 160, 64)
conv2 shape: (None, 160, 160, 64)
pool2 shape: (None, 80, 80, 64)
conv3 shape: (None, 80, 80, 128)
conv3 shape: (None, 80, 80, 128)
pool3 shape: (None, 40, 40, 128)
KerasTensor(type_spec=TensorSpec(shape=(None, 40, 40, 256), dtype=tf.float32, name=None), name='batch_normalization_101/FusedBatchNormV3:0', description="created by layer 'batch_normalization_101'")
KerasTensor(type_spec=TensorSpec(shape=(None, 40, 40, 512), dtype=tf.float32, name=None), name='concatenate_16/concat:0', description="created by layer 'concatenate_16'")
KerasTensor(type_spec=TensorSpec(shape=(None, 40, 40, 128), dtype=tf.float32, name=None), name='conv2d_106/Relu:0', description="created by layer 'conv2d_106'")
KerasTensor(type_spec=TensorSpec(shape=(None, 40, 40, 128), dtype=tf.float32, name=None), name='conv2d_107/Relu:0', description="created by layer 'conv2d_107'")
KerasTe

loading data
load train images...
------------------------------
load test images...
------------------------------
loading data done
conv1 shape: (None, 320, 320, 32)
conv1 shape: (None, 320, 320, 32)
pool1 shape: (None, 160, 160, 32)
conv2 shape: (None, 160, 160, 64)
conv2 shape: (None, 160, 160, 64)
pool2 shape: (None, 80, 80, 64)
conv3 shape: (None, 80, 80, 128)
conv3 shape: (None, 80, 80, 128)
pool3 shape: (None, 40, 40, 128)
KerasTensor(type_spec=TensorSpec(shape=(None, 40, 40, 256), dtype=tf.float32, name=None), name='batch_normalization_123/FusedBatchNormV3:0', description="created by layer 'batch_normalization_123'")
KerasTensor(type_spec=TensorSpec(shape=(None, 40, 40, 512), dtype=tf.float32, name=None), name='concatenate_20/concat:0', description="created by layer 'concatenate_20'")
KerasTensor(type_spec=TensorSpec(shape=(None, 40, 40, 128), dtype=tf.float32, name=None), name='conv2d_129/Relu:0', description="created by layer 'conv2d_129'")
KerasTensor(type_spec=TensorSpec(s

1423/1423 [==============================] - 931s 643ms/step - loss: 0.0648 - batch_ssim: 0.3909 - batch_psnr: 20.3807 - batch_msssim: 0.6709 - batch_nmse: 0.2006 - val_loss: 0.0847 - val_batch_ssim: 0.3409 - val_batch_psnr: 17.3448 - val_batch_msssim: 0.6574 - val_batch_nmse: 0.5117

Epoch 00001: loss improved from inf to 0.06480, saving model to unet.hdf5
Epoch 2/40
1423/1423 [==============================] - 920s 646ms/step - loss: 0.0524 - batch_ssim: 0.4091 - batch_psnr: 21.1434 - batch_msssim: 0.6922 - batch_nmse: 0.1625 - val_loss: 0.0336 - val_batch_ssim: 0.4316 - val_batch_psnr: 23.3135 - val_batch_msssim: 0.7307 - val_batch_nmse: 0.1114

Epoch 00002: loss improved from 0.06480 to 0.05236, saving model to unet.hdf5
Epoch 3/40
1423/1423 [==============================] - 1160s 815ms/step - loss: 0.0482 - batch_ssim: 0.4197 - batch_psnr: 21.3557 - batch_msssim: 0.7031 - batch_nmse: 0.1550 - val_loss: 0.0590 - val_batch_ssim: 0.4263 - val_batch_psnr: 21.3361 - val_batch_msssim: 